## compare the time series of tercile categories derived from the gridded VCSN according to the 2 methods below: 


1) regional averages, then calculation of climatological terciles, then attribution to category   
2) calculation of the tercile categories for each grid point in the region, then calculation of the tercile category for the region is the mode (most frequent category)  

### choose the variable name here 

In [302]:
var_name = 'TMEAN'

### imports 

In [303]:
import pathlib
import sys

In [304]:
%matplotlib inline

In [305]:
from matplotlib import pyplot as plt

In [306]:
import numpy as np

In [307]:
import pandas as pd
import seaborn as sns

### reads the data (csv from probs.lst file)

In [308]:
HOME = pathlib.Path.home()

In [309]:
target_path_freq=  HOME  / 'research' / 'Smart_Ideas' / 'outputs' / 'targets' / 'NZ_regions' / 'NZ_6_regions' / 'CLIOPS' / var_name 
target_path_ave =  HOME  / 'research' / 'Smart_Ideas' / 'outputs' / 'targets' / 'NZ_regions' / 'NZ_6_regions' / var_name

In [310]:
regions = ['NNI','WNI','ENI','NSI','WSI','ESI']

### selects the region here 

In [311]:
matches = {}
disprecancies_dict = {}


for region_name in regions: 
    
    vcsn_freq = pd.read_csv(target_path_freq.joinpath(f'{region_name}/TS_NZ_region_{region_name}_{var_name}_3_quantiles_anoms_cliops.csv'), parse_dates=True, index_col=0)
    vcsn_ave = pd.read_csv(target_path_ave.joinpath(f'{region_name}/TS_NZ_region_{region_name}_{var_name}_3_quantiles_anoms.csv'), parse_dates=True, index_col=0)
    
    match_fraction = (vcsn_ave.cat_3 == vcsn_freq.cat_3).sum() / len(vcsn_ave)
    
    matches[region_name] = match_fraction
    
    disprecancies_freq = vcsn_freq.loc[vcsn_ave.cat_3 != vcsn_freq.cat_3,['1','2','3','cat_3']]
    disprecancies_ave = vcsn_ave.loc[vcsn_ave.cat_3 != vcsn_freq.cat_3,['cat_3']]
    disprecancies_ave.columns = ['cat_3_ave']
    disprecancies_freq.loc[:,'cat_3_ave'] = disprecancies_ave
    
    freq = [] 
    ave = [] 
    for i, row in disprecancies_freq.iterrows(): 
        freq.append(row.loc[['1','2','3']].rank().iloc[int(row.cat_3) - 1])
        ave.append(row.loc[['1','2','3']].rank().iloc[int(row.cat_3_ave) - 1])
    
    freq = np.array(freq)
    ave = np.array(ave)
    
    disprecancies_dict[region_name] = pd.value_counts(ave) / len(ave)

In [312]:
pd.DataFrame(disprecancies_dict) * 100

,NNI,WNI,ENI,NSI,WSI,ESI
1.0,NaN,NaN,NaN,6.25,3.846154,NaN
2.0,100.0,100.0,100.0,93.75,96.153846,100.0


In [313]:
pd.Series(matches)

NNI    0.951020
WNI    0.953061
ENI    0.946939
NSI    0.934694
WSI    0.946939
ESI    0.963265
dtype: float64